<a href="https://colab.research.google.com/github/vitao-bolado/Trabalho2_AnaliseDados/blob/main/Trabalho2_AnaliseDados_VitorFaustinoTirabassi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Trabalho 2 - Análise de Desempenho Estudantil**
Desenvolvido por** Vitor Faustino Tirabassi

O objetivo é elaborar uma análise dos fatores que influenciam o desempenho acadêmico dos estudantes

Desafios:

* Avaliar a influência de sono, motivação e atividades extracurriculares.

* Verificar o impacto do nível socioeconômico.

* Identficar padrões de sucesso escolar.






##**Etapa 1 - Importação das bibliotecas necessárias**

Nesta etapa, serão impotadas as bibliotecas
necessárias. Estas bibliotecas fornecem ferramentas para manipulação de dados (pandas, numpy),
visualização (matplotlib, seaborn) e criação e avaliação de modelos preditivos (sklearn).

In [2]:
# Manipulação de dados
import pandas as pd
import numpy as np

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Modelagem de dados
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier

# Avaliação dos modelos
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Configurações gerais
import warnings
warnings.filterwarnings('ignore')

# Conexão com Google Drive
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive"

Mounted at /content/drive


##**Etapa 2 - Carregamento e visualização inicial dos dados**

Nesta etapa, é carregada a base de dados fornecida e observada sua estrutura, permitindo entender quais variáveis estão presentes, seus tipos e se existem problemas
iniciais como dados ausentes ou colunas desnecessárias.

In [3]:
# Leitura do arquivo CSV
df = pd.read_csv("/content/drive/MyDrive/03_desempenho_estudantil.csv")

# Exibir as primeiras 5 linhas
df.head()

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70


## **Etapa 3 - Análise da estrutura dos dados e tratamento inicial**



**Etapa 3.1 - Verificação da estrutura da base**

Aqui será analisada a dimensão da base de dados (número de registros e variáveis), identificar os tipos de dados presentes
(variáveis categóricas ou numéricas), e verificar se a base está pronta para análise ou necessita de ajustes.

In [4]:
# Quantidade de registros e variáveis
print(f'A base possui {df.shape[0]} registros e {df.shape[1]} variáveis.')

# Tipos de variáveis
print("\nTipos de dados por coluna:\n")
print(df.dtypes)

A base possui 6607 registros e 20 variáveis.

Tipos de dados por coluna:

Hours_Studied                  int64
Attendance                     int64
Parental_Involvement          object
Access_to_Resources           object
Extracurricular_Activities    object
Sleep_Hours                    int64
Previous_Scores                int64
Motivation_Level              object
Internet_Access               object
Tutoring_Sessions              int64
Family_Income                 object
Teacher_Quality               object
School_Type                   object
Peer_Influence                object
Physical_Activity              int64
Learning_Disabilities         object
Parental_Education_Level      object
Distance_from_Home            object
Gender                        object
Exam_Score                     int64
dtype: object


**Etapa 3.2 - Análise de valores ausentes**

Antes da modelagem, é preciso verificar a existência de valores nulos
nas colunas, porque isso pode comprometer a qualidade das previsões e causar erros nos algoritmos.

In [5]:
# Verificando valores nulos por coluna
print("\nValores nulos por coluna:\n")
print(df.isnull().sum().sort_values(ascending=False))


Valores nulos por coluna:

Parental_Education_Level      90
Teacher_Quality               78
Distance_from_Home            67
Hours_Studied                  0
Access_to_Resources            0
Parental_Involvement           0
Attendance                     0
Extracurricular_Activities     0
Motivation_Level               0
Internet_Access                0
Previous_Scores                0
Sleep_Hours                    0
Family_Income                  0
Tutoring_Sessions              0
School_Type                    0
Peer_Influence                 0
Learning_Disabilities          0
Physical_Activity              0
Gender                         0
Exam_Score                     0
dtype: int64


**Etapa 3.3 - Remoção de dados nulos**

Com a confirmação da existência de dados nulos, nesta etapa eles serão removidos para manter a simplicidade e clareza da análise.

In [6]:
# Removendo os registros com valores nulos
df.dropna(inplace=True)

**Etapa 3.4 - Identificação de variáveis categóricas e numéricas**

Ao realizar a identificação e a classificação destas variáveis será possível decidir quais precisam ser codificadas quais precisam ser normalizadas antes de treinar modelos de machine learning.

In [7]:
# Selecionando as variáveis categóricas
cat_cols = df.select_dtypes(include=['object']).columns
print("\nVariáveis categóricas:\n", cat_cols.tolist())

# Selecionando as variáveis numéricas
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
print("\nVariáveis numéricas:\n", num_cols.tolist())



Variáveis categóricas:
 ['Parental_Involvement', 'Access_to_Resources', 'Extracurricular_Activities', 'Motivation_Level', 'Internet_Access', 'Family_Income', 'Teacher_Quality', 'School_Type', 'Peer_Influence', 'Learning_Disabilities', 'Parental_Education_Level', 'Distance_from_Home', 'Gender']

Variáveis numéricas:
 ['Hours_Studied', 'Attendance', 'Sleep_Hours', 'Previous_Scores', 'Tutoring_Sessions', 'Physical_Activity', 'Exam_Score']


##**Etapa 4 - Codificação das variáveis categóricas**

Os algoritmos de machine learning do scikit-learn exigem que todas as variáveis estejam em formato numérico.
Como algumas variáveis da base de dados são categóricas, precisamos codificá-las, usando o LabelEncoder. Essa técnica atribui um número inteiro diferente para cada categoria, permitindo o uso nos modelos.

In [9]:
# Instanciação do codificador
le = LabelEncoder()

# Aplicando o LabelEncoder para as colunas categóricas
for col in cat_cols:
    df[col] = le.fit_transform(df[col])

# Verificação do resultado
df.head()


,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,1,0,0,7,73,1,1,0,1,2,1,2,3,0,1,2,1,67
1,19,64,1,2,0,8,59,1,1,2,2,2,1,0,4,0,0,1,0,61
2,24,98,2,2,1,7,91,2,1,2,2,2,1,1,4,0,2,2,1,74
3,29,89,1,2,1,8,98,2,1,1,2,2,1,0,4,0,1,1,1,71
4,19,92,2,2,1,6,65,2,1,3,2,0,1,1,4,0,0,2,0,70


##**Etapa 5 - Normalização das variáveis numéricas**

A normalização garante que todas as variáveis numéricas estejam na mesma escala. É importante para algoritmos baseados em KNN e SVM e ajuda na convergência mais rápida de outros modelos. Será usado o StandardScaler, que transforma os dados para uma distribuição com média 0 e desvio padrão 1.

In [10]:
# Removendo a variável alvo
num_cols_to_scale = num_cols.drop('Exam_Score')

# Instanciando o normalizador
scaler = StandardScaler()

# Aplicação da normalização nos dados numéricos
df[num_cols_to_scale] = scaler.fit_transform(df[num_cols_to_scale])

# Visualização do resultado
df[num_cols_to_scale].describe().T

,count,mean,std,min,25%,50%,75%,max
Hours_Studied,6378.0,-1.693360e-16,1.000078,-3.170784,-0.664514,0.003825,0.672163,4.013856
Attendance,6378.0,5.458858e-16,1.000078,-1.733435,-0.867620,-0.001805,0.864009,1.729824
Sleep_Hours,6378.0,-1.030499e-16,1.000078,-2.067530,-0.705056,-0.023819,0.657418,2.019892
Previous_Scores,6378.0,4.634459e-16,1.000078,-1.740795,-0.837971,-0.004595,0.898229,1.731605
Tutoring_Sessions,6378.0,1.002647e-17,1.000078,-1.211858,-0.401411,-0.401411,0.409036,5.271718
Physical_Activity,6378.0,8.021179e-17,1.000078,-2.889375,-0.945447,0.026516,0.998480,2.942408


##**Etapa 6 - Separação dos dados (Hold-out)**

**Etapa 6.1 - Criação da variável alvo para classificação**

Para a aplicação de algoritmos de classificação, é preciso transformar a variável numérica 'Exam_Score' em categórica.
A ideia é agrupar os valores em faixas que representam o desempenho dos alunos em Ruim, Regular, Bom e Excelente.

In [12]:
# Criando a variável Score_Class baseada nas faixas de nota
df['Score_Class'] = pd.cut(df['Exam_Score'],
                           bins=[0, 50, 70, 85, 100],
                           labels=['Ruim', 'Regular', 'Bom', 'Excelente'])

# Removendo registros sem classificação, por prevenção
df.dropna(subset=['Score_Class'], inplace=True)

# Codificando a nova coluna
df['Score_Class'] = le.fit_transform(df['Score_Class'])

**Etapa 6.2 - Separação das variáveis preditivas (x) e alvo (y) para a classificação**

Para o treinamento dos modelos de classificação, devemos remover as variáveis alvo do DataFrame e separar
os dados em X (entradas) e y (rótulo/classificação).

In [14]:
X_class = df.drop(columns=['Exam_Score', 'Score_Class'])
y_class = df['Score_Class']

**Etapa 6.3 - Separação das variáveis preditoras (X) e alvo (y) para a regressão**

Para a regressão, a variável alvo será a própria nota (Exam_Score), que é uma variável contínua. A remoção da coluna Score_Class será feita para evitar a influência da classificação nos modelos de regressão.

In [15]:
X_reg = df.drop(columns=['Exam_Score', 'Score_Class'])
y_reg = df['Exam_Score']

**Etapa 6.4 - Aplicação da divisão Houd-out**

O Hold-out é uma técnica de validação simples consistindo em separar os dados em dois subconjuntos: treino, para construir os modelos, e teste, para avaliação.
A proporção padrão usada será de 70% para treino e 30% para teste.

In [16]:
# Separação dos dados para classificação
Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_class, y_class, test_size=0.3, random_state=42)

# Separação os dados para regressão
Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_reg, y_reg, test_size=0.3, random_state=42)


##**Etapa 7 - Aplicação dos modelos de classificação**

Agora serão aplicados os cinco algoritmos de classificação solicitados no enunciado do trabalho:
Regressão Logística, Árvore de Decisão, Random Forest, SVM e KNN.
Cada modelo será avaliado com as principais métricas de classificação: Acurácia, Precisão, Revocação (Recall) e F1-Score.
O conjunto de teste, à parte dos dados não vistos pelo modelo, será utilizado para garantir uma avaliação justa.

**Etapa 7.1 - Regressão Logística**

In [17]:
print("\nREGRESSÃO LOGÍSTICA")
log_model = LogisticRegression()
log_model.fit(Xc_train, yc_train)
yc_pred_log = log_model.predict(Xc_test)

print("Acurácia:", accuracy_score(yc_test, yc_pred_log))
print("Precisão:", precision_score(yc_test, yc_pred_log, average='macro'))
print("Recall:", recall_score(yc_test, yc_pred_log, average='macro'))
print("F1-Score:", f1_score(yc_test, yc_pred_log, average='macro'))


REGRESSÃO LOGÍSTICA
Acurácia: 0.9284221525600836
Precisão: 0.5906164075236635
Recall: 0.5638238083768451
F1-Score: 0.5758931742311244


**Etapa 7.2 - Árvore de Decisão**

In [18]:
print("\nÁRVORE DE DECISÃO")
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(Xc_train, yc_train)
yc_pred_tree = tree_model.predict(Xc_test)

print("Acurácia:", accuracy_score(yc_test, yc_pred_tree))
print("Precisão:", precision_score(yc_test, yc_pred_tree, average='macro'))
print("Recall:", recall_score(yc_test, yc_pred_tree, average='macro'))
print("F1-Score:", f1_score(yc_test, yc_pred_tree, average='macro'))


ÁRVORE DE DECISÃO
Acurácia: 0.8605015673981191
Precisão: 0.5074275439873283
Recall: 0.5147293562438509
F1-Score: 0.5107629804257395


**Etapa 7.3 - Random Forest**

In [20]:
print("\nRANDOM FOREST")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(Xc_train, yc_train)
yc_pred_rf = rf_model.predict(Xc_test)

print("Acurácia:", accuracy_score(yc_test, yc_pred_rf))
print("Precisão:", precision_score(yc_test, yc_pred_rf, average='macro'))
print("Recall:", recall_score(yc_test, yc_pred_rf, average='macro'))
print("F1-Score:", f1_score(yc_test, yc_pred_rf, average='macro'))


RANDOM FOREST
Acurácia: 0.9242424242424242
Precisão: 0.6122362820394188
Recall: 0.5304378118795579
F1-Score: 0.5604153842375498


**Etapa 7.4 - SVM (kernel linear)**

In [21]:
print("\nSVM")
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(Xc_train, yc_train)
yc_pred_svm = svm_model.predict(Xc_test)

print("Acurácia:", accuracy_score(yc_test, yc_pred_svm))
print("Precisão:", precision_score(yc_test, yc_pred_svm, average='macro'))
print("Recall:", recall_score(yc_test, yc_pred_svm, average='macro'))
print("F1-Score:", f1_score(yc_test, yc_pred_svm, average='macro'))


SVM
Acurácia: 0.9263322884012539
Precisão: 0.5858962647178237
Recall: 0.5638721169639346
F1-Score: 0.5739481114678372


**Etapa 7.5 - KNN (K=5)**

In [30]:
print("\nKNN")
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(Xc_train, yc_train)
yc_pred_knn = knn_model.predict(Xc_test)

print("Acurácia:", accuracy_score(yc_test, yc_pred_knn))
print("Precisão:", precision_score(yc_test, yc_pred_knn, average='macro'))
print("Recall:", recall_score(yc_test, yc_pred_knn, average='macro'))
print("F1-Score:", f1_score(yc_test, yc_pred_knn, average='macro'))


KNN
Acurácia: 0.8892371995820272
Precisão: 0.5684438243418457
Recall: 0.4697887621979182
F1-Score: 0.4989667797600992


##**Etapa 8 - Aplicação dos modelos de regressão**

Nesta etapa serão aplicados os quatro algoritmos de regressão: Regressão Linear, Árvore de Regressão, Random Forest Regressor e SVR (kernel linear). O objetivo é prever diretamente a nota dos estudantes (Exam_Score), que é um valor contínuo. Para avaliar o desempenho de cada modelo, as métricas utilizadas serão: Erro Médio Absoluto (MAE) e Raiz do Erro Quadrático Médio (RMSE).

**Etapa 8.1 - Regressão Linear**

In [32]:
print("\nREGRESSÃO LINEAR")
lin_model = LinearRegression()
lin_model.fit(Xr_train, yr_train)
yr_pred_lin = lin_model.predict(Xr_test)

mae_lin = mean_absolute_error(yr_test, yr_pred_lin)
rmse_lin = np.sqrt(mean_squared_error(yr_test, yr_pred_lin))

print("MAE:", mae_lin)
print("RMSE:", rmse_lin)


REGRESSÃO LINEAR
MAE: 1.036219515016636
RMSE: 2.097675271255808


Etapa 8.2 - Árvore de Regressão

In [36]:
print("\nÁRVORE DE REGRESSÃO")
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(Xr_train, yr_train)
yr_pred_tree = tree_reg.predict(Xr_test)

mae_tree = mean_absolute_error(yr_test, yr_pred_tree)
rmse_tree = np.sqrt(yr_test, yr_pred_tree)

print("MAE:", mae_tree)
print("RMSE:", rmse_tree)


ÁRVORE DE REGRESSÃO
MAE: 1.910135841170324
RMSE: 2813    8.246211
4421    8.124038
3190    8.000000
1246    8.485281
4699    8.185353
          ...   
1115    8.306624
746     8.124038
6260    8.185353
6497    8.062258
1565    8.000000
Name: Exam_Score, Length: 1914, dtype: float64


Etapa 8.3 - Random Forest Regressor

In [38]:
print("\nRANDOM FOREST REGRESSOR")
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg.fit(Xr_train, yr_train)
yr_pred_rf = rf_reg.predict(Xr_test)

mae_rf = mean_absolute_error(yr_test, yr_pred_rf)
rmse_rf = np.sqrt(yr_test, yr_pred_rf)

print("MAE:", mae_rf)
print("RMSE:", rmse_rf)


RANDOM FOREST REGRESSOR
MAE: 1.1676384535005224
RMSE: 2813    8.246211
4421    8.124038
3190    8.000000
1246    8.485281
4699    8.185353
          ...   
1115    8.306624
746     8.124038
6260    8.185353
6497    8.062258
1565    8.000000
Name: Exam_Score, Length: 1914, dtype: float64


**Etapa 8.4 - SVR (kernel linear)**

In [42]:
print("\nSVR")
svr_model = SVR(kernel='linear', C=1.0)
svr_model.fit(Xr_train, yr_train)
yr_pred_svr = svr_model.predict(Xr_test)

mae_svr = mean_absolute_error(yr_test, yr_pred_svr)
rmse_svr = np.sqrt(yr_test, yr_pred_svr)

print("MAE:", mae_svr)
print("RMSE:", rmse_svr)


SVR
MAE: 1.0292468395801548
RMSE: 2813    8.246211
4421    8.124038
3190    8.000000
1246    8.485281
4699    8.185353
          ...   
1115    8.306624
746     8.124038
6260    8.185353
6497    8.062258
1565    8.000000
Name: Exam_Score, Length: 1914, dtype: float64
